In [4]:
import pandas as pd
import numpy as np
import glob
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('default')
import sys
sys.path.append('..')
  
# %matplotlib inline: only draw static
# images in the notebookº
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Carga de información

In [22]:
laliga2122 = pd.read_excel('../data/LigasEspanolas6T/LaLiga21 22.xlsx', ).dropna(how='all')
laliga2021 = pd.read_excel('../data/LigasEspanolas6T/LaLiga20 21.xlsx', ).dropna(how='all')
laliga1920 = pd.read_excel('../data/LigasEspanolas6T/LaLiga19 20.xlsx', ).dropna(how='all')
laliga1819 = pd.read_excel('../data/LigasEspanolas6T/LaLiga18 19.xlsx', ).dropna(how='all')
laliga1718 = pd.read_excel('../data/LigasEspanolas6T/LaLiga17 18.xlsx', ).dropna(how='all')
laliga1617 = pd.read_excel('../data/LigasEspanolas6T/LaLiga16 17.xlsx', ).dropna(how='all')
seg2122 =  pd.read_excel('../data/LigasEspanolas6T/LaLiga123 21 22.xlsx', ).dropna(how='all')
seg2021 =  pd.read_excel('../data/LigasEspanolas6T/LaLiga123 20 21.xlsx', ).dropna(how='all')
seg1920 =  pd.read_excel('../data/LigasEspanolas6T/LaLiga123 19 20.xlsx', ).dropna(how='all')
seg1819 =  pd.read_excel('../data/LigasEspanolas6T/LaLiga123 18 19.xlsx', ).dropna(how='all')
seg1718 =  pd.read_excel('../data/LigasEspanolas6T/LaLiga123 17 18.xlsx', ).dropna(how='all')
seg1617 =  pd.read_excel('../data/LigasEspanolas6T/LaLiga123 16 17.xlsx', ).dropna(how='all')


## Creando dataframes

In [28]:
dfs_liga = [laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617]
dfs_segunda = [seg2122, seg2021, seg1920, seg1819, seg1718, seg1617]
cols = ['Wk','Date','Local','Score','Visitante', 'Attendance']

laliga = pd.concat(dfs_liga)[cols]
segunda = pd.concat(dfs_segunda)[cols]

laliga['Attendance']= laliga['Attendance']*1000
laliga['Torneo'] = 'Primera división'
segunda['Attendance']= segunda['Attendance']*1000
segunda['Torneo'] = 'Segunda división'

partidos6t = pd.concat([laliga,segunda], axis= 0).fillna({'Attendance': 0})
partidos6t

,Wk,Date,Local,Score,Visitante,Attendance,Torneo
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,Primera división
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,Primera división
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,Primera división
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,Primera división
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,Primera división
...,...,...,...,...,...,...,...
495,42.0,2017-06-10,Valladolid,1–0,Cádiz,12812.0,Segunda división
496,42.0,2017-06-10,Gimnàstic,1–0,UCAM Murcia,11103.0,Segunda división
497,42.0,2017-06-10,Alcorcón,3–0,Lugo,4262.0,Segunda división
498,42.0,2017-06-10,Córdoba,2–1,Girona,8977.0,Segunda división


## Creando nuevas columnas

In [29]:
def golesvisita(marcador):
    goles = marcador.split('–')
    return int(goles[1])
def goleslocal(marcador):
    goles = marcador.split('–')
    return int(goles[0])

partidos6t['Goles Local'] = partidos6t['Score'].apply(goleslocal)
partidos6t['Goles Visitante'] = partidos6t['Score'].apply(golesvisita)
partidos6t['Total goles partido']= partidos6t['Goles Local']+partidos6t['Goles Visitante']


In [30]:
def torneo(año,mes):
    if ((año==2016) & (mes<=12)|(año==2017) & (mes<8)):
        return '16/17'
    elif ((año==2017) & (mes<=12)|(año==2018) & (mes<8)):
        return '17/18'
    elif ((año==2018) & (mes<=12)|(año==2019) & (mes<8)):
        return '18/19'
    elif ((año==2019) & (mes<=12)|(año==2020) & (mes<9)):
        return '19/20'
    elif ((año==2020) & (mes<=12)|(año==2021) & (mes<8)):
        return '20/21'
    elif ((año==2021) & (mes<=12)|(año==2022) & (mes<8)):
        return '21/22'

    
partidos6t['Año'] = partidos6t['Date'].dt.year
partidos6t['Mes'] = partidos6t['Date'].dt.month
partidos6t['Temporada'] = partidos6t.apply(lambda x: torneo(x['Año'],x['Mes']),axis = 1)


In [31]:
def resultado(string):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return 'Local'
    if int(goles[0])<int(goles[1]):
        return 'Visitante'
    else: 
        return 'Empate'
    
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    
partidos6t['Resultado'] = partidos6t['Score'].apply(resultado)
partidos6t['Ganador'] = partidos6t.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)



In [45]:
partidos6t

,Wk,Date,Local,Score,Visitante,Attendance,Torneo,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,Primera división,1,0,1,2021,8,21/22,Local,Valencia
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,Primera división,1,1,2,2021,8,21/22,Empate,Empate
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,Primera división,1,1,2,2021,8,21/22,Empate,Empate
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,Primera división,0,0,0,2021,8,21/22,Empate,Empate
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,Primera división,1,4,5,2021,8,21/22,Visitante,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,42.0,2017-06-10,Valladolid,1–0,Cádiz,12812.0,Segunda división,1,0,1,2017,6,16/17,Local,Valladolid
496,42.0,2017-06-10,Gimnàstic,1–0,UCAM Murcia,11103.0,Segunda división,1,0,1,2017,6,16/17,Local,Gimnàstic
497,42.0,2017-06-10,Alcorcón,3–0,Lugo,4262.0,Segunda división,3,0,3,2017,6,16/17,Local,Alcorcón
498,42.0,2017-06-10,Córdoba,2–1,Girona,8977.0,Segunda división,2,1,3,2017,6,16/17,Local,Córdoba


In [49]:
temporada = '21/22'
equipo = 'Betis'
torneo = 'Primera división'
desde = 20
hasta = 25

def aplicar_condiciones(conditions: list) -> bool:
    if not conditions:
        return True
    return conditions[0] & aplicar_condiciones(conditions[1:])

condicion_general = [
    partidos6t['Wk'] >= desde,
    partidos6t['Wk'] <= hasta,
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada
]



df = partidos6t[aplicar_condiciones(condicion_general)]

condicion_visitante = [df['Visitante'] == equipo]
condicion_local = [df['Local'] == equipo]

goles_visitante = df[aplicar_condiciones(condicion_visitante)]['Goles Visitante'].sum()
goles_local = df[aplicar_condiciones(condicion_local)]['Goles Local'].sum()
print(equipo, 'en la temporada', temporada, 'tiene', goles_visitante + goles_local, 'puntos.')

Betis en la temporada 21/22 tiene 15 puntos.


## Tabla de posiciones hasta la fecha 33 (inclusive)

In [50]:
ej2122 = partidos6t[(partidos6t['Temporada']=='21/22')&(partidos6t['Torneo']=='Primera división')]
equip2122 = ej2122['Local'].unique().tolist()

In [106]:
def tabla_posicion(data, fecha, equipos):
    """
    input = data[['Wk', 'Local', 'Visitante', 'Ganador']]
    output = df[['Equipo', 'Puntos']]
    """
    tabla = {}
    for equipo in equipos:
        data_usar = data[(data['Wk']<=fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
        puntos = 0
        for i in range(len(data_usar)):
            ganador = str(data_usar.iloc[i]['Ganador'])
            if ganador == equipo:
                puntos += 3
            if ganador == 'Empate':
                puntos +=1
        tabla[equipo] = puntos
    df = pd.DataFrame([[key, tabla[key]] for key in tabla.keys()], columns=['Equipo', 'Puntos'])
    df = df.sort_values(by='Puntos',ascending = False)
    return df

def partidos_rest(data, equipo, fecha):
    """
    input = data[['Wk', 'Local', 'Visitante']]
    output = df['partido restantes']
    """
    rest = data[(data['Wk']>fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
    return rest

def primera_est(data, tabla, fecha, equipo):
    """
    input1 = data[['Wk', 'Local', 'Visitante']] # Partidos Restantes
    input2 = tabla[['Puntos']] # Puntos acumulados a la fecha
    output = Puntaje estimado de fin
    """
    dato = data[(data['Wk']>fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
    indice = tabla[tabla['Equipo']==equipo].index.values.astype(int)[0]
    puntos = tabla.loc[indice,'Puntos']
    for i in range(len(dato)):
        if dato.iloc[i]['Local']==equipo:
            rival = dato.iloc[i]['Visitante']
        else:
            rival = dato.iloc[i]['Local']
        index = tabla[tabla['Equipo']==rival].index.values.astype(int)[0]
        puntos_rival = tabla.loc[index,'Puntos']
        if puntos<puntos_rival:
            puntos+=3
        if puntos == puntos_rival:
            puntos +=1
    return puntos

def segunda_est(data, tabla, fecha, equipo):
    """
    input1 = data[['Wk', 'Local', 'Visitante']] # Partidos Restantes
    input2 = tabla[['Puntos']] # Puntos acumulados a la fecha
    output = Puntaje estimado de fin
    """
    dato = data[(data['Wk']>fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
    indice = tabla[tabla['Equipo']==equipo].index.values.astype(int)[0]
    puntos = tabla.loc[indice,'Puntos']
    for i in range(len(dato)):
        if dato.iloc[i]['Local']==equipo:
            rival = dato.iloc[i]['Visitante']
        else:
            rival = dato.iloc[i]['Local']
        index = tabla[tabla['Equipo']==rival].index.values.astype(int)[0]
        puntos_rival = tabla.loc[index,'Puntos']
        if puntos>puntos_rival:
            puntos+=3
        if puntos == puntos_rival:
            puntos +=1
    return puntos

def calcular_goles(data, fecha_rango, equipos, goles_recibidos=True):
    desde = fecha_rango[0]
    hasta = fecha_rango[1]
    condiciones = aplicar_condiciones([data['Wk'] >= desde, data['Wk'] <= hasta])
    df = data[condiciones]
    resultado = {}
    
    for equipo in equipos:
        df_equipo = df[(df['Visitante']==equipo) | (df['Local']==equipo)]

        total_goles = 0
        for i in range(len(df_equipo)):
            local = str(df_equipo.iloc[i]['Local'])

            if goles_recibidos:

                if local == equipo:
                    goles = df_equipo.iloc[i]['Goles Visitante']
                else:
                    goles = df_equipo.iloc[i]['Goles Local']
            
            else:
                if local == equipo:
                    goles = df_equipo.iloc[i]['Goles Local']
                else:
                    goles = df_equipo.iloc[i]['Goles Visitante']

            total_goles += goles

        resultado[equipo] = total_goles

    name_columna = 'Goles Recibidos' if goles_recibidos else 'Goles Realizados'
    df = pd.DataFrame([[key, resultado[key]] for key in resultado.keys()], columns=['Equipo', name_columna])
       
    return df


In [60]:
tabla = tabla_posicion(ej2122,33,equip2122)
tabla.head()

,Equipo,Puntos
19,Real Madrid,78
6,Barcelona,63
7,Sevilla,63
15,Atlético Madrid,61
10,Betis,57


In [67]:
tabla_final = tabla_posicion(ej2122,38,equip2122)
tabla_final.head(7)

,Equipo,Puntos
19,Real Madrid,86
6,Barcelona,73
15,Atlético Madrid,71
7,Sevilla,70
10,Betis,65
14,Real Sociedad,62
8,Villarreal,59


## Partidos restantes

In [68]:
partidos_rest(ej2122,'Valencia',33)

,Wk,Date,Local,Score,Visitante,Attendance,Torneo,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
373,34.0,2022-04-30,Valencia,1–1,Levante,35099.0,Primera división,1,1,2,2022,4,21/22,Empate,Empate
383,35.0,2022-05-07,Athletic Club,0–0,Valencia,41091.0,Primera división,0,0,0,2022,5,21/22,Empate,Empate
392,36.0,2022-05-10,Valencia,0–3,Betis,28091.0,Primera división,0,3,3,2022,5,21/22,Visitante,Betis
403,37.0,2022-05-14,Espanyol,1–1,Valencia,14990.0,Primera división,1,1,2,2022,5,21/22,Empate,Empate
416,38.0,2022-05-21,Valencia,2–0,Celta Vigo,16189.0,Primera división,2,0,2,2022,5,21/22,Local,Valencia


## Primera estimación

In [57]:
tabla[tabla['Equipo']=='Valencia'] 

,Equipo,Puntos
0,Valencia,42


In [116]:
tabla

,Equipo,Puntos
19,Real Madrid,78
6,Barcelona,63
7,Sevilla,63
15,Atlético Madrid,61
10,Betis,57
14,Real Sociedad,55
8,Villarreal,52
13,Athletic Club,48
3,Osasuna,44
0,Valencia,42


In [114]:
primera_est(ej2122,tabla,33,'Valencia')

48

## Segunda estimación

In [79]:
tabla[tabla['Equipo']=='Valencia']

,Equipo,Puntos
0,Valencia,42


In [83]:
segunda_est(ej2122,tabla,33,'Valencia')

51

# Test Sumativo

In [139]:
temporada = '21/22'
torneo = 'Primera división'
equipos = ['Mallorca']
hasta = 17

condicion_general = aplicar_condiciones([
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada,
])

df = partidos6t[condicion_general]

tabla_posicion(df, hasta, equipos).loc[0,'Puntos']

20

In [140]:
temporada = '21/22'
torneo = 'Primera división'
equipos = ['Betis']
hasta = [19, 25]

condicion_general = aplicar_condiciones([
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada,
])

df = partidos6t[condicion_general]

tabla_posicion(df, hasta[1], equipos).loc[0,'Puntos'] - tabla_posicion(df, hasta[0], equipos).loc[0,'Puntos']

13

In [ ]:
temporada = '21/22'
torneo = 'Primera división'
equipos = ['Osasuna', 'Sevilla']
fecha_rango = [1, 24]

condiciones = aplicar_condiciones([
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada
])

calcular_goles(data=partidos6t[condiciones], fecha_rango=fecha_rango, equipos=equipos)

,Equipo,Goles Recibidos
0,Osasuna,27
1,Sevilla,16


In [105]:
temporada = '21/22'
torneo = 'Primera división'
equipos = ['Elche', 'Real Sociedad']
fecha_rango = [1, 20]

condiciones = aplicar_condiciones([
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada
])

calcular_goles(data=partidos6t[condiciones], fecha_rango=fecha_rango, equipos=equipos, goles_recibidos=False)

,Equipo,Goles Realizados
0,Elche,20
1,Real Sociedad,22


In [ ]:
temporada = '21/22'
torneo = 'Primera división'
desde = 30

condiciones = aplicar_condiciones([
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada,
    partidos6t['Wk'] >= desde,
])

partidos6t[condiciones]['Attendance'].sum()

2392319.0

In [138]:
temporada = '21/22'
torneo = 'Primera división'
equipos = partidos6t['Local'].unique()
equipos_target = ['Getafe', 'Mallorca', 'Granada', 'Espanyol']
hasta = 33

condiciones = aplicar_condiciones([
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada
])

tablaWk33 = tabla_posicion(data=partidos6t[condiciones], fecha=hasta, equipos=equipos)

resultado = {}
for equipo in equipos_target:
    resultado[equipo] =  primera_est(
        data=partidos6t[condiciones], 
        tabla=tablaWk33, 
        fecha=33, 
        equipo=equipo
        )

df_estimacion = pd.DataFrame([[key, resultado[key]] for key in resultado.keys()], columns=['Equipo', 'Puntos Estimados'])

df_estimacion

temporada = '21/22'
torneo = 'Primera división'
equipos = partidos6t['Local'].unique()
equipos_target = ['Getafe', 'Mallorca', 'Granada', 'Espanyol']
hasta = 38

condiciones = aplicar_condiciones([
    partidos6t['Torneo'] == torneo,
    partidos6t['Temporada'] == temporada
])

df_real = tabla_posicion(data=partidos6t[condiciones], fecha=hasta, equipos=equipos_target)
df_real

df = df_estimacion.merge(df_real)
df['diff'] = abs(df['Puntos Estimados']-df['Puntos'])
df.sort_values(by='diff', ascending=False)


,Equipo,Puntos Estimados,Puntos,diff
0,Getafe,48,39,9
1,Mallorca,45,39,6
3,Espanyol,45,42,3
2,Granada,40,38,2


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 0 to 423
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Wk                   380 non-null    float64       
 1   Date                 380 non-null    datetime64[ns]
 2   Local                380 non-null    object        
 3   Score                380 non-null    object        
 4   Visitante            380 non-null    object        
 5   Attendance           380 non-null    float64       
 6   Torneo               380 non-null    object        
 7   Goles Local          380 non-null    int64         
 8   Goles Visitante      380 non-null    int64         
 9   Total goles partido  380 non-null    int64         
 10  Año                  380 non-null    int32         
 11  Mes                  380 non-null    int32         
 12  Temporada            380 non-null    object        
 13  Resultado            380 non-null    obj

In [147]:
df.loc[0, 'Wk'] = None
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 0 to 423
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Wk                   379 non-null    float64       
 1   Date                 380 non-null    datetime64[ns]
 2   Local                380 non-null    object        
 3   Score                380 non-null    object        
 4   Visitante            380 non-null    object        
 5   Attendance           380 non-null    float64       
 6   Torneo               380 non-null    object        
 7   Goles Local          380 non-null    int64         
 8   Goles Visitante      380 non-null    int64         
 9   Total goles partido  380 non-null    int64         
 10  Año                  380 non-null    int32         
 11  Mes                  380 non-null    int32         
 12  Temporada            380 non-null    object        
 13  Resultado            380 non-null    obj

In [152]:
str.upper(df['Visitante'])

TypeError: descriptor 'upper' for 'str' objects doesn't apply to a 'Series' object